In [7]:
# List of candidates in the order listed on the voting site.
candidates = ["Alexandre Eichenberger (IBM)",	"Saurabh Tangri (Intel Corporation)",	"Jeremy (Landscaping)",	
              "Mayank Kaushik (NVIDIA Corporation)",	"Prasanth Pulavarthi (Microsoft)",	"Andreas Fehlner (TRUMPF Laser GmbH)",	
              "Christian Bourjau (QuantCo)"]
# 4 letter identifier to enable simple identification in verbose results.
candidate_short = ["Alex",	"Saur",	"Jere",	"Maya",	" Pras", "Andr", "Chri"]


# Votes with company (added manually), 
#   name (replaced manually from email address, which should be avoided in published docs), 
#   and votes ranking for each candidate.
# Raw info can be pulled from the voting site as cvs file.
votes = [
["MS","Jacky Chen",1,2,2,2,1,1,2],
["IBM","Tung LeDuc",1,4,5,3,2,5,5],
["NVIDIA","Joaquin Anton",3,4,7,1,2,5,6],
["MS","Nataniel MvVicar",2,5,7,3,1,4,6],
["AMD","Maximilian Bartel",1,6,4,7,5,2,3],
["IBM","Alexandre Eichenberger",2,2,7,2,1,2,7],
["MIT","Tian Jin",1,7,7,7,7,7,7],
["AMD","Dominik Montada",1,5,7,4,2,6,3],
["NVIDIA","Mayank Kaushik",3,5,7,1,2,4,6],
["IBM","Charles Volzka",1,3,7,4,2,6,5],
["INTEL","Yuwen Zhou",4,1,6,3,2,7,5],
["GROQ","Philip Lassen",1,5,7,3,2,4,6],
["INTEL","Ettore Tiotto",1,2,5,4,3,6,7],
["IBM","Michael Essenmacher",1,5,7,4,2,3,6],
["NVIDIA","Yuan Yao",5,7,7,1,6,7,7],
["Preferred Networks","Zijian Xu",2,4,7,3,1,5,6],
["INTEL","Mengni Wang",3,1,3,2,2,3,3],
["INTEL","Whitney Tsang",1,2,7,7,7,7,7],
["INTEL","Feng Tian",4,1,6,3,2,7,5],
["TRUMPF","Andreas Fehlner",3,5,7,4,3,1,6],
["IBM","Chin Huang",1,4,7,3,2,7,7],
["MS","Liqun Fu",2,4,7,3,1,5,7],
["GROQ","Soren Lassen",1,4,7,3,2,5,6],
["NVIDIA","Zhenhua Wang",2,7,4,1,3,5,6],
["MS","Justin Chu",2,3,7,3,1,4,4],
["INTEL","Haihao Shen",7,2,3,6,1,5,4],
["GROQ","Roberto DiCecco",2,1,7,3,4,5,6],
["Preferred Networks","Takeshi Watanabe",2,6,4,1,3,5,7],
["MS","Prasanth Pulavarthi",2,2,7,2,1,2,3],
["QUANTCO","Jakub Bachurski",1,5,7,5,2,4,3],
["ARM","Eric Kunze",1,3,7,3,2,4,6],
["INTEL","Saurabh Tangri",1,1,7,7,1,1,7],
["MS","Ashay Rane",3,5,5,2,1,3,4],
["MS","G. Ramalingam",1,2,7,2,1,2,2],
["MS","Wei-Sheng Chin",1,2,4,2,1,3,3],
["IBM","Tong Chen",1,7,7,7,2,7,7],
["INTEL","Hengyu Meng",3,1,4,6,2,5,7],
["IBM","Yasushi Negishi",1,7,7,7,7,7,7],
["GROQ","Ram Krishna",3,1,4,4,2,4,4],
["NVIDIA","Rajeev Rao",3,4,7,1,2,6,5],
["AMD","Philipp Braun",1,7,7,7,2,7,7],
["IBM","Haruki Imai",1,2,7,2,2,2,2],
]

In [8]:
import numpy as np

# From the list of all ballots, seclect the ones for this company (or all if company is "ALL").
def ballot_for_company(ballots, select_company, verbose=0):
    res = []
    for b in ballots:
        curr_company, curr_ballot = b[0], b[2:]
        if select_company == 'ALL' or curr_company == select_company:
            res.append(curr_ballot)
            if verbose:
                print(curr_company, curr_ballot)
    return np.array(res)

# Help for debugging.
def justified_numbers(a):
    return map(lambda n: str(n).rjust(4), a)

def print_justified_numbers(title, a):
    print(title)
    print("  ", *candidate_short)
    print("  ", *justified_numbers(a))

# Create a new ballot reflecting the choices in the given ballot sample. 
# Verbose of 1 print debugging info, verbose of 2 additionally print every ballot used.
def rank(ballots, candidates, verbose=0):
    cnum = candidates.size
    num_ballots = 0
    #m[a,b] is "time folks preferred candidate b over a".
    m = np.zeros((cnum, cnum), dtype=int)
    for b in ballots:
        num_ballots += 1
        for j in range(cnum):
            for k in range(cnum):
                if b[k]>b[j]:
                    m[k][j] = m[k][j] + 1
    total = np.sum(m, axis=0) # sum over columns
    # (candidate id, preference score) ranked by most preferred first
    rank = sorted(zip(range(cnum),total), reverse=True, key=lambda record: record[1])
    rank_named = sorted(zip(candidate_short,total), reverse=True, key=lambda record: record[1])
    # New ballot: where candidates that have the same preferrence have the same score (low is better).
    new_ballot = np.zeros(cnum, dtype=int)
    preference = 0
    previous_score = 10000000
    for p in range(cnum): # By order of decreasing popularity.
        curr_candidate_id, curr_candidate_score = rank[p]
        if curr_candidate_score != previous_score:
            preference +=1 # Different score, advance by 1.
        new_ballot[curr_candidate_id] = preference
        previous_score = curr_candidate_score   
    if verbose:
        print("##########################################################")
        print("Tabulating", num_ballots, "votes.")
        if verbose > 1:
            i = 1
            for b in ballots:
                print("  ", i, ":", b)
                i += 1
        print("Matrix m[a, b], times voters preferred candidate b over a:")
        print("          ", *candidate_short, sep = " ")
        for i in range(cnum):
            m_i_str = justified_numbers(m[i])
            print("  ", i, candidate_short[i], ":", *m_i_str, sep = " ")
        print_justified_numbers("Totals (sum over columns of m)", total)
        print_justified_numbers("Rank (candidate id, candidate preference score)",rank_named)
        print_justified_numbers("New ballot", new_ballot)
    return (num_ballots, new_ballot)

# Create a ballot that reflects all the votes per company.
# Verbose of 1 print debugging info, verbose of 2 additionally print every ballot used.
def rank_per_company(votes, candidates, verbose=0):
    companies = np.unique(votes[:, 0])
    ballots_per_company = []
    ballots_per_company_with_name = []
    (tot_num_ballots, counted_num_ballot) = (np.shape(votes)[0], 0)
    for com in companies:
        print("Ballot for company = ", com)
        (company_num_ballots, new_ballot) = rank(ballot_for_company(votes, com), candidates, verbose=verbose)
        counted_num_ballot += company_num_ballots
        print("  is ", new_ballot, " with ", company_num_ballots, "counted ballots\n")
        ballots_per_company.append(new_ballot)
        ballots_per_company_with_name.append([com, new_ballot])
    assert counted_num_ballot == tot_num_ballots
    print("Final ballot with one vote per company")
    (final_num_ballot, final_ballot) = rank(np.array(ballots_per_company), candidates, verbose=verbose)
    print("  is ", final_ballot, " with ", final_num_ballot, "counted ballots\n")
    assert final_num_ballot == np.shape(companies)[0]
    if verbose:
        print("Final ballot = ", final_ballot)
        print("")
    return (counted_num_ballot, final_ballot, ballots_per_company_with_name)

# Translate a ballot into the list of ordered candidates using names.
def ballot_to_candidates(ballot, candidates):
    cnum = candidates.size
    # (candidate id, order) ranked by most preferred first
    rank = sorted(zip(range(cnum), ballot), key=lambda record: record[1])
    for r in rank:
        num = str(r[1])+". "
        print(num, candidates[r[0]])

In [9]:
# Use numpy arrays.
vv = np.array(votes)
cc = np.array(candidates)
verbose = 1 # 0 none, 1 some, 2 print ballots also.
#ballot_for_company(vv, 'INTEL')
#rank(ballot_for_company(vv, "INTEL"), cc, verbose=1)

print("Non-official vote where each voter get one voice. This is not the offical result and is only provided to investgate the difference between individual votes vs votes per company\n")
(total_votes1, ballot_per_voter) = rank(ballot_for_company(vv, "ALL"), cc, verbose=verbose)
print("\nBallot per voter =", ballot_per_voter)
ballot_to_candidates(ballot_per_voter, cc)
print("\n\n")

print("Official vote per our election rules, where votes are tallied by company first, and the final round includes exactly one ballot per company\n")
(total_votes2, final_ballot_per_company, ballots_per_company) = rank_per_company(vv, cc, verbose=verbose)
assert total_votes1 == total_votes2
print("\nFinal ballot (summarizing per company)=", final_ballot_per_company)
ballot_to_candidates(final_ballot_per_company, cc)



Non-official vote where each voter get one voice. This is not the offical result and is only provided to investgate the difference between individual votes vs votes per company

##########################################################
Tabulating 42 votes.
Matrix m[a, b], times voters preferred candidate b over a:
           Alex Saur Jere Maya  Pras Andr Chri
   0 Alex :    0    7    1   11   16    2    1
   1 Saur :   32    0    3   17   30   11    5
   2 Jere :   40   32    0   31   38   25   23
   3 Maya :   29   12    3    0   30    8    4
   4  Pras :   21    8    1    7    0    2    1
   5 Andr :   34   21    7   24   34    0    7
   6 Chri :   40   29    5   28   37   21    0
Totals (sum over columns of m)
   Alex Saur Jere Maya  Pras Andr Chri
    196  109   20  118  185   69   41
Rank (candidate id, candidate preference score)
   Alex Saur Jere Maya  Pras Andr Chri
   ('Alex', 196) (' Pras', 185) ('Maya', 118) ('Saur', 109) ('Andr', 69) ('Chri', 41) ('Jere', 20)
New ballot
 

In [10]:
def printResultMD(year, ballots, aggregated_ballots, candidates, candidate_shortname, results, verbose=0):
    cnum = candidates.size
    # Table of votes
    print("# Vote results for year " + year + ".")
    print("## Votes casted by Contributors.")
    str = "| Contributors | "
    for c in candidates:
        str += c + " | "
    print(str)
    str = "|"
    for c in range(cnum+1):
        str += "-------|"
    print(str)
    for b in ballots:
        str = "| " + b[1] + " (" + b[0] + ") | "
        for c in range(cnum):
            str += b[c+2] + " | "
        print(str)
    # Print ballots per company
    print("\n## Precedence per company.")
    print("| Member company | Perference (Numbers are indices of columns from vote table) |")
    print("|----------------|-------------------------------------------------------------|")
    for b in aggregated_ballots:
        bb = np.array(b[1])
        rank = sorted(zip(candidate_shortname,bb), key=lambda record: record[1])
        if (verbose):
            print("ballot:", b)
            print("rank ", rank)
        str = "| " + b[0] + " | "
        for c in range(cnum):
            if c>0:
                if rank[c][1] == rank[c-1][1]:
                    str += " = "
                else:
                    str += " > "
            str += rank[c][0]
        print(str+ " |")        
    # Print final results
    print("\n## Final results (Schultz method, top 5 are elected).")
    ballot_to_candidates(results, candidates)

In [6]:
# Copy output in a separate file for easy consumption of the results.
printResultMD("2023-2024", vv, ballots_per_company, cc, candidate_short, final_ballot_per_company)

# Vote results for year 2023-2024.
## Votes casted by Contributors.
| Contributors | Alexandre Eichenberger (IBM) | Saurabh Tangri (Intel Corporation) | Jeremy (Landscaping) | Mayank Kaushik (NVIDIA Corporation) | Prasanth Pulavarthi (Microsoft) | Andreas Fehlner (TRUMPF Laser GmbH) | Christian Bourjau (QuantCo) | 
|-------|-------|-------|-------|-------|-------|-------|-------|
| Jacky Chen (MS) | 1 | 2 | 2 | 2 | 1 | 1 | 2 | 
| Tung LeDuc (IBM) | 1 | 4 | 5 | 3 | 2 | 5 | 5 | 
| Joaquin Anton (NVIDIA) | 3 | 4 | 7 | 1 | 2 | 5 | 6 | 
| Nataniel MvVicar (MS) | 2 | 5 | 7 | 3 | 1 | 4 | 6 | 
| Maximilian Bartel (AMD) | 1 | 6 | 4 | 7 | 5 | 2 | 3 | 
| Alexandre Eichenberger (IBM) | 2 | 2 | 7 | 2 | 1 | 2 | 7 | 
| Tian Jin (MIT) | 1 | 7 | 7 | 7 | 7 | 7 | 7 | 
| Dominik Montada (AMD) | 1 | 5 | 7 | 4 | 2 | 6 | 3 | 
| Mayank Kaushik (NVIDIA) | 3 | 5 | 7 | 1 | 2 | 4 | 6 | 
| Charles Volzka (IBM) | 1 | 3 | 7 | 4 | 2 | 6 | 5 | 
| Yuwen Zhou (INTEL) | 4 | 1 | 6 | 3 | 2 | 7 | 5 | 
| Philip Lassen (GROQ)